In [ ]:
# Extract FTSE100 data
#date extrated from (https://uk.finance.yahoo.com/quote/%5EFTSE/) & (https://uk.finance.yahoo.com/crypto/)
#I chose Yahoo finance as it is a public domine and accesable without the need to login 
#and create a personal API key

# Importing library

In [1]:
import yfinance as yf #gets data from yahoo finance public accesable domain 
import pandas as pd #pandas for data manipulation
from datetime import datetime, timedelta #use to convert data into usable date time format
import matplotlib.pyplot as plt #for plotting 
import seaborn as sns# for plotting

### Define the timeframe

In [2]:
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

### Extract Bitcoin GBP data

In [3]:
bit_gb = yf.download('BTC-GBP', start=start_date, end=end_date)
bit_gb.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [4]:
bit_gb.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-07-16,8705.041992,8855.687500,7607.077148,7636.543945,7636.543945,19459659592
1,2019-07-17,7631.430664,8012.960449,7383.736328,7798.092285,7798.092285,19765052261
2,2019-07-18,7800.389160,8631.723633,7510.553223,8505.368164,8505.368164,20083931958
3,2019-07-19,8495.379883,8541.433594,8173.279297,8423.533203,8423.533203,16579868305
4,2019-07-20,8419.602539,8837.515625,8359.975586,8612.333008,8612.333008,16162705677


In [5]:
# Cleaning Bitcoin data
bit_gb.dropna(inplace=True) #dropping null values
bit_gb['Date'] = pd.to_datetime(bit_gb['Date'])

In [6]:
bit_gb.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-07-16,8705.041992,8855.687500,7607.077148,7636.543945,7636.543945,19459659592
1,2019-07-17,7631.430664,8012.960449,7383.736328,7798.092285,7798.092285,19765052261
2,2019-07-18,7800.389160,8631.723633,7510.553223,8505.368164,8505.368164,20083931958
3,2019-07-19,8495.379883,8541.433594,8173.279297,8423.533203,8423.533203,16579868305
4,2019-07-20,8419.602539,8837.515625,8359.975586,8612.333008,8612.333008,16162705677


## Feature Engineering: 

#### Exponential Moving Average for BitCoin

In [7]:
#Calculate 20, 50, 200 Exponential Moving Average
bit_gb['20_MA'] = bit_gb['Adj Close'].rolling(window=20).mean()
bit_gb['50_MA'] = bit_gb['Adj Close'].rolling(window=50).mean()
bit_gb['200_MA'] = bit_gb['Adj Close'].rolling(window=200).mean()

In [25]:
bit_gb.head()

,Date,Open,High,Low,Close,Adj Close,Volume,20_MA,50_MA,200_MA
0,2019-07-16,8705.041992,8855.687500,7607.077148,7636.543945,7636.543945,19459659592,NaN,NaN,NaN
1,2019-07-17,7631.430664,8012.960449,7383.736328,7798.092285,7798.092285,19765052261,NaN,NaN,NaN
2,2019-07-18,7800.389160,8631.723633,7510.553223,8505.368164,8505.368164,20083931958,NaN,NaN,NaN
3,2019-07-19,8495.379883,8541.433594,8173.279297,8423.533203,8423.533203,16579868305,NaN,NaN,NaN
4,2019-07-20,8419.602539,8837.515625,8359.975586,8612.333008,8612.333008,16162705677,NaN,NaN,NaN


### Extract FTSE100 data
date extrated from (https://uk.finance.yahoo.com/quote/%5EFTSE/) & (https://uk.finance.yahoo.com/crypto/)I chose Yahoo finance as it is a public domine and accesable without the need to login  and create a personal API key

In [31]:
ftse100 = yf.download('^FTSE', start=start_date, end=end_date)
ftse100.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [32]:
ftse100.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-07-16,7531.700195,7590.100098,7524.500000,7577.200195,7577.200195,607182100
1,2019-07-17,7577.200195,7577.200195,7519.200195,7535.500000,7535.500000,576654300
2,2019-07-18,7535.500000,7535.500000,7482.600098,7493.100098,7493.100098,504046800
3,2019-07-19,7493.100098,7546.700195,7475.500000,7508.700195,7508.700195,575615700
4,2019-07-22,7508.700195,7549.700195,7491.799805,7514.899902,7514.899902,513122700


In [33]:
# Cleaning FTSE100 data
ftse100.dropna(inplace=True) #dropping null values
ftse100['Date'] = pd.to_datetime(ftse100['Date'])

### Calculate Moving Averages for FTSE100

#### The SMA is the arithmetic mean of the closing prices over a specified number of periods.
#### It provides a smoothed line that helps to identify the trend direction.
#### A longer SMA (e.g., 30-day) is smoother and less sensitive to short-term price fluctuations than a shorter SMA (e.g., 7-day)


In [ ]:
#Exponential Moving Average (EMA):
#The EMA gives more weight to recent prices, making it more responsive to new information compared to the SMA.
#The span parameter controls how quickly the weights decay for older observations.
#The 7-day EMA responds more quickly to price changes than the 30-day EMA due to its shorter span

In [34]:
#calculate 7 days and 30 days exponential moving average (EMA)
ftse100['FTSE100_7Day_EMA'] =ftse100['Close'].ewm(span=7, adjust=False).mean()
ftse100['FTSE100_30Day_EMA'] =ftse100['Close'].ewm(span=30, adjust=False).mean()

#pandas .ewm() was used as it is a type of moving average to calculate \
#the exponentially weighted moving average for a certain number of previous periods.

In [35]:
#calculate 7 and 30 day simple moving averages(SMA)
ftse100['FTSE100_7Day_SMA'] = ftse100['Close'].rolling(window=7).mean()
ftse100['FTSE100_30Day_SMA'] = ftse100['Close'].rolling(window=30).mean()

In [36]:
ftse100.head()

,Date,Open,High,Low,Close,Adj Close,Volume,FTSE100_7Day_EMA,FTSE100_30Day_EMA,FTSE100_7Day_SMA,FTSE100_30Day_SMA
0,2019-07-16,7531.700195,7590.100098,7524.500000,7577.200195,7577.200195,607182100,7577.200195,7577.200195,NaN,NaN
1,2019-07-17,7577.200195,7577.200195,7519.200195,7535.500000,7535.500000,576654300,7566.775146,7574.509860,NaN,NaN
2,2019-07-18,7535.500000,7535.500000,7482.600098,7493.100098,7493.100098,504046800,7548.356384,7569.257617,NaN,NaN
3,2019-07-19,7493.100098,7546.700195,7475.500000,7508.700195,7508.700195,575615700,7538.442337,7565.350687,NaN,NaN
4,2019-07-22,7508.700195,7549.700195,7491.799805,7514.899902,7514.899902,513122700,7532.556728,7562.095798,NaN,NaN


In [37]:
### Merge the bitcoin and FTSE100 data on the Date column

In [38]:
# Merge the data on the Date column
data = pd.merge(ftse100, bit_gb[['Date', 'Close']], on='Date', how='inner')

In [39]:
data.head()

,Date,Open,High,Low,Close_x,Adj Close,Volume,FTSE100_7Day_EMA,FTSE100_30Day_EMA,FTSE100_7Day_SMA,FTSE100_30Day_SMA,Close_y
0,2019-07-16,7531.700195,7590.100098,7524.500000,7577.200195,7577.200195,607182100,7577.200195,7577.200195,NaN,NaN,7636.543945
1,2019-07-17,7577.200195,7577.200195,7519.200195,7535.500000,7535.500000,576654300,7566.775146,7574.509860,NaN,NaN,7798.092285
2,2019-07-18,7535.500000,7535.500000,7482.600098,7493.100098,7493.100098,504046800,7548.356384,7569.257617,NaN,NaN,8505.368164
3,2019-07-19,7493.100098,7546.700195,7475.500000,7508.700195,7508.700195,575615700,7538.442337,7565.350687,NaN,NaN,8423.533203
4,2019-07-22,7508.700195,7549.700195,7491.799805,7514.899902,7514.899902,513122700,7532.556728,7562.095798,NaN,NaN,8293.061523


In [42]:
data.rename(columns={'Close_x': 'FTSE100_Close', 'Close_y': 'Bitcoin_Price_GBP'}, inplace=True)

In [43]:
data.head()

,Date,Open,High,Low,FTSE100_Close,Adj Close,Volume,FTSE100_7Day_EMA,FTSE100_30Day_EMA,FTSE100_7Day_SMA,FTSE100_30Day_SMA,Bitcoin_Price_GBP
0,2019-07-16,7531.700195,7590.100098,7524.500000,7577.200195,7577.200195,607182100,7577.200195,7577.200195,NaN,NaN,7636.543945
1,2019-07-17,7577.200195,7577.200195,7519.200195,7535.500000,7535.500000,576654300,7566.775146,7574.509860,NaN,NaN,7798.092285
2,2019-07-18,7535.500000,7535.500000,7482.600098,7493.100098,7493.100098,504046800,7548.356384,7569.257617,NaN,NaN,8505.368164
3,2019-07-19,7493.100098,7546.700195,7475.500000,7508.700195,7508.700195,575615700,7538.442337,7565.350687,NaN,NaN,8423.533203
4,2019-07-22,7508.700195,7549.700195,7491.799805,7514.899902,7514.899902,513122700,7532.556728,7562.095798,NaN,NaN,8293.061523


In [ ]:
# percentage change------.pct_change() calculates the percentage change between 
#the current and previous closing prices

In [44]:
data['FTSE100_Return'] = data['FTSE100_Close'].pct_change()
data['Bitcoin_Return'] = data['Bitcoin_Price_GBP'].pct_change()

In [45]:
data.head()

,Date,Open,High,Low,FTSE100_Close,Adj Close,Volume,FTSE100_7Day_EMA,FTSE100_30Day_EMA,FTSE100_7Day_SMA,FTSE100_30Day_SMA,Bitcoin_Price_GBP,FTSE100_Return,Bitcoin_Return
0,2019-07-16,7531.700195,7590.100098,7524.500000,7577.200195,7577.200195,607182100,7577.200195,7577.200195,NaN,NaN,7636.543945,NaN,NaN
1,2019-07-17,7577.200195,7577.200195,7519.200195,7535.500000,7535.500000,576654300,7566.775146,7574.509860,NaN,NaN,7798.092285,-0.005503,0.021155
2,2019-07-18,7535.500000,7535.500000,7482.600098,7493.100098,7493.100098,504046800,7548.356384,7569.257617,NaN,NaN,8505.368164,-0.005627,0.090699
3,2019-07-19,7493.100098,7546.700195,7475.500000,7508.700195,7508.700195,575615700,7538.442337,7565.350687,NaN,NaN,8423.533203,0.002082,-0.009622
4,2019-07-22,7508.700195,7549.700195,7491.799805,7514.899902,7514.899902,513122700,7532.556728,7562.095798,NaN,NaN,8293.061523,0.000826,-0.015489


In [47]:
data.shape

(1260, 14)

In [48]:
# Drop rows with NaN values created by rolling and pct_change
data.dropna(inplace=True)

In [49]:
data.shape

(1231, 14)

In [50]:
# Display the first few rows of the merged data
data.head()

,Date,Open,High,Low,FTSE100_Close,Adj Close,Volume,FTSE100_7Day_EMA,FTSE100_30Day_EMA,FTSE100_7Day_SMA,FTSE100_30Day_SMA,Bitcoin_Price_GBP,FTSE100_Return,Bitcoin_Return
29,2019-08-27,7095.000000,7110.799805,7044.700195,7089.600098,7089.600098,851399100,7133.016819,7297.776151,7135.528669,7347.116699,8292.351562,-0.000761,-0.021773
30,2019-08-28,7089.600098,7131.200195,7050.000000,7114.700195,7114.700195,675638900,7128.437663,7285.964799,7135.171526,7331.700033,7983.849121,0.003540,-0.037203
31,2019-08-29,7114.700195,7199.200195,7103.100098,7184.299805,7184.299805,635681800,7142.403199,7279.405767,7134.400042,7319.993359,7804.203613,0.009783,-0.022501
32,2019-08-30,7184.299805,7240.399902,7179.700195,7207.200195,7207.200195,727868800,7158.602448,7274.747343,7146.142927,7310.463363,7893.874023,0.003188,0.011490
33,2019-09-02,7207.200195,7315.299805,7206.899902,7281.899902,7281.899902,497764900,7189.426811,7275.208798,7157.271484,7302.903353,8575.850586,0.010365,0.086393


In [51]:
# Summary Statistics
print("\nSummary Statistics:\n")
data.describe()


Summary Statistics:



,Open,High,Low,FTSE100_Close,Adj Close,Volume,FTSE100_7Day_EMA,FTSE100_30Day_EMA,FTSE100_7Day_SMA,FTSE100_30Day_SMA,Bitcoin_Price_GBP,FTSE100_Return,Bitcoin_Return
count,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1.231000e+03,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000
mean,7174.327053,7219.121778,7129.565238,7175.219254,7175.219254,8.435573e+08,7172.633095,7164.678294,7172.605862,7164.175225,23722.513072,0.000185,0.002163
std,628.434904,617.739488,641.207523,629.209957,629.209957,3.220907e+08,619.883198,588.319457,622.350882,597.493087,13585.263502,0.011097,0.040032
min,4993.899902,5181.000000,4898.799805,4993.899902,4993.899902,0.000000e+00,5246.886851,5803.318270,5175.571429,5530.920036,3954.907959,-0.108738,-0.359284
25%,6960.500000,7007.949951,6900.500000,6960.500000,6960.500000,6.428026e+08,6975.792926,6991.180583,6965.628557,6997.670036,11865.009277,-0.004283,-0.016043
50%,7324.500000,7368.299805,7282.500000,7326.100098,7326.100098,7.763260e+08,7328.603601,7313.064306,7325.285645,7316.686637,22547.867188,0.000659,0.001099
75%,7573.300049,7613.400146,7532.300049,7573.600098,7573.600098,9.731642e+08,7567.021528,7532.492527,7564.178571,7529.021647,32963.210938,0.005252,0.019995
max,8445.799805,8474.400391,8427.900391,8445.799805,8445.799805,2.813498e+09,8409.093244,8239.025561,8429.414202,8300.103320,57090.710938,0.090530,0.210290


In [56]:
# Save data as CSV
csv_file_path = 'final_data_AXA.csv'
data.to_csv(csv_file_path, index=False)